In [15]:
import requests
import csv
import hashlib
import json


In [5]:

AD_ACCOUNT_ID = ""
API_VERSION = "v11.0"


In [6]:
ACCESS_TOKEN = ""



In [25]:
# These are some helper functions to hash some match keys.
# Each Field may have a different hashing format
# https://developers.facebook.com/docs/marketing-api/audiences/guides/custom-audiences#hash

def hashEmail(email:str) -> str:
    return hashlib.sha256(email.strip().lower().encode()).hexdigest()

def hashName(name:str) -> str:
    return hashlib.sha256(name.strip().lower().encode()).hexdigest()

In [26]:
# setup some sample data

SAMPLE_DATA = [
    # ("first name", "last name", "email")
    ("john", "smith", "john@smith.com"),
    ("will", "smith", "will@smith.com"),
    ("will", "ferrell", "will@ferrell.com"),
]

In [42]:
# Pretty print the sample data hashed

sampleJsonHashed = [{
    entry[0]: hashName(entry[0]),
    entry[1]: hashName(entry[1]),
    entry[2]: hashEmail(entry[2]),
} for entry in SAMPLE_DATA]

print(json.dumps(sampleJsonHashed, indent=4))

[
    {
        "john": "96d9632f363564cc3032521409cf22a852f2032eec099ed5967c0d000cec607a",
        "smith": "6627835f988e2c5e50533d491163072d3f4f41f5c8b04630150debb3722ca2dd",
        "john@smith.com": "7e27ef2718f1aaddb03936077714a60244e380ef09e27504fcb3184a2af7b6ea"
    },
    {
        "will": "bc16f5825a9dafc8c245dc7cd93f4e671a9885035a73e6ef29c04f2d3863bfab",
        "smith": "6627835f988e2c5e50533d491163072d3f4f41f5c8b04630150debb3722ca2dd",
        "will@smith.com": "a2ef9d51cc534717139d46cd152c3a1847b4bdba8f52f72fdac9af42eb4ab194"
    },
    {
        "will": "bc16f5825a9dafc8c245dc7cd93f4e671a9885035a73e6ef29c04f2d3863bfab",
        "ferrell": "f7969d794271cf4450491c034ec297cd44881f73e775dcda36ba50db82860f11",
        "will@ferrell.com": "cd25f26fd21ac317d555d8d4c738118a2f784cad1efa15a56f600e89f846abd5"
    }
]


In [8]:
# Step 1: Create the Custom Audience ID
# Just need to do this once to create the ID
# Or can use the UI to create and store the CUSTOM_AUDIENCE_ID

params = {
    "name": "UK_2+_CV_123456789",
    "subtype": "CUSTOM",
    "description": "People who purchased on my website",
    "customer_file_source": "BOTH_USER_AND_PARTNER_PROVIDED",
    "access_token": ACCESS_TOKEN
}
url = f"https://graph.facebook.com/{API_VERSION}/act_{AD_ACCOUNT_ID}/customaudiences"
resp = requests.post(url, params=params)
print(resp.json())
CUSTOM_AUDIENCE_ID = resp.json()['id']
print(CUSTOM_AUDIENCE_ID)

{'id': '23848244085250141', '__fb_trace_id__': 'GvIiAqVCEh+', '__www_request_id__': 'AjkE4E0Bgv3DNVt0iCSqVQY'}
23848244085250141


In [35]:
# Step 2: Upload the audience data to the CUSTOM_AUDIENCE_ID
# Make sure to match the data ordering to the schema and that the data is hashed.

params = {
    "payload": json.dumps({
        "schema": [
            "FN",
            "LN",
            "EMAIL",
        ],
        "data": [
            [
                hashName(entry[0]),
                hashName(entry[1]),
                hashEmail(entry[2]),
            ] for entry in SAMPLE_DATA
        ],
    }),
    "access_token": ACCESS_TOKEN
}
url = f"https://graph.facebook.com/{API_VERSION}/{CUSTOM_AUDIENCE_ID}/users"
resp = requests.post(url, params=params)
print(resp.json())

{'audience_id': '23848244085250141', 'session_id': '2878562040898028798', 'num_received': 3, 'num_invalid_entries': 0, 'invalid_entry_samples': {}, '__fb_trace_id__': 'GckQn568gPf', '__www_request_id__': 'AEwmeApJ1EzRmNVkc44ifFs'}
